In [1]:
from transformers import pipeline
import scipy
import torch

device = 1; # torch.cuda.device(1)
#torch.cuda.set_device(1)

MAX_TOKENS: int = 1503 # 30 seconds, (about 50 tokens/second)

time_tokens = lambda t: t * 50

TIME: int = 5

In [2]:
rate, data = scipy.io.wavfile.read('sounds/mists.wav')
data = torch.Tensor(data)[rate * TIME:, :]#.cuda(1)
print(rate, data.shape)

44100 torch.Size([374850, 2])


In [3]:
from transformers import AutoProcessor, MusicgenMelodyForConditionalGeneration
import torch


processor = AutoProcessor.from_pretrained("facebook/musicgen-melody", low_cpu_mem_usage=True)
print('processor complete')
model = MusicgenMelodyForConditionalGeneration.from_pretrained("facebook/musicgen-melody").cuda(1)
print('model complete')

print('preprocessing')

inputs = processor(
    audio=data,
    sampling_rate=rate,
    text=["classical piano"],
    padding=True,
    return_tensors="pt",
)

print('postprocessing')

for k in inputs.keys():
    inputs[k] = inputs[k].to(f'cuda:1')

processor complete


/home/evanmm3/research/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/evanmm3/research/venv/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model complete
preprocessing


: 

In [ ]:
audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=time_tokens(TIME), use_cache=True)
print('generated')

In [4]:
import soundfile as sf

sampling_rate = model.config.audio_encoder.sampling_rate
sf.write("sounds/output_cache.wav", audio_values[0].T.cpu().numpy(), sampling_rate)

# post-run
#del model
#torch.cuda.empty_cache()

other stuff

In [ ]:
synthesiser = pipeline("text-to-audio", "facebook/musicgen-small", device=1)

music = synthesiser("lo-fi music with a soothing melody", forward_params={"do_sample": True})

scipy.io.wavfile.write("sounds/musicgen_out.wav", rate=music["sampling_rate"], data=music["audio"])

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs, max_new_tokens=256)

In [ ]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].numpy(), rate=sampling_rate)